In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
#import pymongo
import pandas as pd
from splinter import Browser
from datetime import datetime

In [2]:
import json

# MARS NEWS

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [4]:
#Check status code (ALWAYS)
response = requests.get(url)
response.status_code

200

In [5]:
# Confirm we snagged the html
html = response.text
html[0:50]

"<!DOCTYPE html>\n<html lang='en' xml:lang='en' xmln"

In [6]:
# SOUPIFY
soup = BeautifulSoup(html, "lxml")
#print(soup.prettify())

In [7]:
#grab all articles (from class)
titles = soup.find_all(class_="content_title")

#confirm length
len(titles)

6

In [8]:
firstTitle = titles[0].a.text.strip()
firstTitle

"The Launch Is Approaching for NASA's Next Mars Rover, Perseverance"

In [9]:
firstLink = "https://mars.nasa.gov" + titles[0].a["href"]
firstLink

'https://mars.nasa.gov/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/'

In [10]:
#grab all articles (from class)
texts = soup.find_all(class_="article_teaser_body")

#confirm length
len(texts)

0

In [11]:
#TRAGIC

In [12]:
#we have to use splinter for this page apparently

In [13]:
executable_path = {'executable_path': r"/home/bdr/Desktop/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [15]:
html = browser.html

In [16]:
soup = BeautifulSoup(html, "lxml")
#print(soup.prettify())

In [17]:
#grab all articles (from class)
titles = soup.find_all(class_="content_title")

#confirm length
len(titles)

51

In [18]:
firstTitle = "" 
for title in titles:
    if title.a:
        firstTitle = title
        break

In [19]:
newsTitle = firstTitle.a.text.strip()
newsTitle

"How NASA's Mars Helicopter Will Reach the Red Planet's Surface"

In [20]:
newsLink = "https://mars.nasa.gov" + firstTitle.a['href']
newsLink

'https://mars.nasa.gov/news/8699/how-nasas-mars-helicopter-will-reach-the-red-planets-surface/'

In [21]:
#grab all articles (from class)
texts = soup.find_all(class_="article_teaser_body")

#confirm length
len(texts)

40

In [30]:
newsText = texts[0].text.strip()
newsText

'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.'

In [29]:
browser.quit()

In [24]:
def scrapeMarsInfo():
    #Have to use Splinter as this is a dynamic URL
    executable_path = {'executable_path': r"/home/bdr/Desktop/chromedriver"}
    browser = Browser('chrome', **executable_path, headless=False) #don't load a browser window
    
    #visit URL
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    
    #soupify
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    
    #Grap the classes we want
    titles = soup.find_all(class_="content_title")
    texts = soup.find_all(class_="article_teaser_body")
    
    # Find the first title with an actual link (that is an a tag). The first thing with this class is not an article title sadly.
    firstTitle = "" 
    for title in titles:
        if title.a: #ugh
            firstTitle = title
            break
    
    # Parse information we want
    newsTitle = firstTitle.a.text.strip()
    newsLink = "https://mars.nasa.gov" + firstTitle.a['href']
    newsText = texts[0].text.strip()
    
    # NAVIGATE TO NEXT WEBSITE HERE
    #visit URL
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    
    #grab images
    images = soup.find_all(class_="carousel_item")
    imageURL = 'https://www.jpl.nasa.gov' + images[0]["style"].split(" ")[1].split("'")[1]
    
    # NAVIGATE TO NEXT WEBSITE HERE
    url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    
    #Get Weather
    allTweets_Maybe = soup.find_all("span")
    tweetText = ""
    for tweet in allTweets_Maybe:
        if tweet.text:
            if "InSight sol" in tweet.text:
                tweetText = tweet.text
                break
                
    allLinks_Maybe = soup.find_all("a")
    tweetLink = ""
    for link in allLinks_Maybe:
        if link['href']:
            if "status" in link["href"]:
                tweetLink = "https://www.twitter.com" + link["href"]
                break
    
    # NAVIGATE TO NEXT WEBSITE HERE
    url = 'https://space-facts.com/mars/'
    browser.visit(url)
    html = browser.html
    dfs = pd.read_html(html)
    stats = dfs[0]
    stats.columns = ["Attribute", "Value"]
    
    #format and save
    data_html = stats.to_html(index=False)
    data_stats = json.loads(stats.to_json(orient="records"))

    # GET NEW INFORMATION HERE    
    
    # close the browser
    browser.quit()
    
    # create rnt dictionary
    rtnDict = {
        "newsTitle": newsTitle,
        "newsLink": newsLink,
        "newsText": newsText,
        "featureImageURL": imageURL,
        "tweetWeatherURL": tweetLink,
        "tweetWeatherText": tweetText,
        "marsStatsHTML": data_html,
        "marsStats": data_stats,
        "dateScraped": datetime.now()
    }
    
    return rtnDict
    

In [32]:
marsNews = scrapeMarsInfo()

In [33]:
marsNews

{'newsTitle': "How NASA's Mars Helicopter Will Reach the Red Planet's Surface",
 'newsLink': 'https://mars.nasa.gov/news/8699/how-nasas-mars-helicopter-will-reach-the-red-planets-surface/',
 'newsText': 'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.',
 'featureImageURL': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18284-1920x1200.jpg',
 'tweetWeatherURL': '',
 'tweetWeatherText': '',
 'marsStatsHTML': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Attribute</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</t

# Featured Image

In [ ]:
executable_path = {'executable_path': r"/home/bdr/Desktop/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False) #don't load a browser window

#visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [ ]:
#soupify
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [ ]:
images = soup.find_all(class_="carousel_item")
len(images)

In [ ]:
imageURL = images[0]["style"].split(" ")[1].split("'")[1]
'https://www.jpl.nasa.gov' + imageURL

# Twitter Weather


In [ ]:
executable_path = {'executable_path': r"/home/bdr/Desktop/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False) #don't load a browser window

#visit URL
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [ ]:
#soupify
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [ ]:
ughhh = soup.find_all("span")

In [ ]:
len(ughhh)

In [ ]:
tweetText = ""

for tweet in ughhh:
    if tweet.text:
        if "InSight sol" in tweet.text:
            tweetText = tweet.text
            break
            
print(tweetText)

In [ ]:
allLinks_Maybe = soup.find_all("a")

len(allLinks_Maybe)

In [ ]:
tweetLink = ""

for link in allLinks_Maybe:
    if link['href']:
        if "status" in link["href"]:
            tweetLink = "https://www.twitter.com" + link["href"]
            break

print(tweetLink)

# Mars Facts

In [ ]:
executable_path = {'executable_path': r"/home/bdr/Desktop/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False) #don't load a browser window

#visit URL
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [ ]:
#soupify
html = browser.html

In [ ]:
dfs = pd.read_html(html)

In [ ]:
len(dfs)

In [ ]:
stats = dfs[0]
stats.columns = ["Attribute", "Value"]
stats

In [ ]:
json.loads(stats.to_json(orient="records"))